In [1]:
USE Sales
GO

Commands completed successfully.

Total execution time: 00:00:00.004

In [2]:
CREATE EXTERNAL FILE FORMAT csv_file
WITH (
    FORMAT_TYPE = DELIMITEDTEXT,
    FORMAT_OPTIONS(
        FIELD_TERMINATOR = ',',
        STRING_DELIMITER = '"',
        FIRST_ROW = 2,
        USE_TYPE_DEFAULT = TRUE)
);

: Msg 46502, Level 16, State 1, Line 1
Type with name 'csv_file' already exists.

Total execution time: 00:00:00.076

In [3]:
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
BEGIN
  CREATE EXTERNAL DATA SOURCE SqlStoragePool
  WITH (LOCATION = 'sqlhdfs://controller-svc/default');
END

Commands completed successfully.

Total execution time: 00:00:00.008

In [4]:
CREATE EXTERNAL TABLE [web_clickstreams_hdfs]
("wcs_click_date_sk" BIGINT , "wcs_click_time_sk" BIGINT , "wcs_sales_sk" BIGINT , "wcs_item_sk" BIGINT , "wcs_web_page_sk" BIGINT , "wcs_user_sk" BIGINT)
WITH
(
    DATA_SOURCE = SqlStoragePool,
    LOCATION = '/clickstream_data',
    FILE_FORMAT = csv_file
);
GO

: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'web_clickstreams_hdfs' in the database.

Total execution time: 00:00:00.072

In [6]:
SELECT  
    wcs_user_sk,
    SUM( CASE WHEN i_category = 'Books' THEN 1 ELSE 0 END) AS book_category_clicks,
    SUM( CASE WHEN i_category_id = 1 THEN 1 ELSE 0 END) AS [Home & Kitchen],
    SUM( CASE WHEN i_category_id = 2 THEN 1 ELSE 0 END) AS [Music],
    SUM( CASE WHEN i_category_id = 3 THEN 1 ELSE 0 END) AS [Books],
    SUM( CASE WHEN i_category_id = 4 THEN 1 ELSE 0 END) AS [Clothing & Accessories],
    SUM( CASE WHEN i_category_id = 5 THEN 1 ELSE 0 END) AS [Electronics],
    SUM( CASE WHEN i_category_id = 6 THEN 1 ELSE 0 END) AS [Tools & Home Improvement],
    SUM( CASE WHEN i_category_id = 7 THEN 1 ELSE 0 END) AS [Toys & Games],
    SUM( CASE WHEN i_category_id = 8 THEN 1 ELSE 0 END) AS [Movies & TV],
    SUM( CASE WHEN i_category_id = 9 THEN 1 ELSE 0 END) AS [Sports & Outdoors]
  FROM [dbo].[web_clickstreams_hdfs]
  INNER JOIN item it ON (wcs_item_sk = i_item_sk
                        AND wcs_user_sk IS NOT NULL)
GROUP BY  wcs_user_sk;
GO

(88695 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:18.267

wcs_user_sk,book_category_clicks,Home & Kitchen,Music,Books,Clothing & Accessories,Electronics,Tools & Home Improvement,Toys & Games,Movies & TV,Sports & Outdoors
9529,6,1,18,6,2,1,3,1,1,1
65271,35,3,53,35,4,7,4,4,5,2
31624,49,7,75,49,12,6,2,4,15,7
67193,18,1,15,18,1,3,2,1,4,1
32549,133,10,216,133,16,18,14,6,30,13
39326,12,2,12,12,0,0,0,0,3,2
94965,9,1,14,9,0,1,1,2,6,1
25368,7,0,17,7,1,2,0,1,3,1
86522,13,1,14,13,0,1,0,3,5,1
29727,123,16,181,123,19,21,10,8,27,15
